In [1]:
import os
import sys
import random
os.environ["KERAS_BACKEND"] = "tensorflow"

import glob
try:
    if not ("CUDA_VISIBLE_DEVICES" in os.environ):
        os.environ['CUDA_VISIBLE_DEVICES']='0'
        print("importing setGPU")
        import setGPU
except:
    print("Could not import setGPU, please make sure you configure CUDA_VISIBLE_DEVICES manually")
    pass

import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas
import time
from tqdm import tqdm
import itertools
import io
import sklearn
import sklearn.cluster
import tensorflow as tf
from numpy.lib.recfunctions import append_fields

import scipy
import scipy.special

from mpnn import MessagePassing, ReadoutGraph, Aggregation

importing setGPU
Could not import setGPU, please make sure you configure CUDA_VISIBLE_DEVICES manually


In [2]:
import tensorflow as tf
import json, os
import numpy as np

# Tested with TensorFlow 2.1.0
print('version={}, CUDA={}, GPU={}, TPU={}'.format(
    tf.__version__, tf.test.is_built_with_cuda(),
    # GPU attached?
    len(tf.config.list_physical_devices('GPU')) > 0,
    # TPU accessible? (only works on Colab)
    'COLAB_TPU_ADDR' in os.environ))

version=2.1.0, CUDA=True, GPU=True, TPU=False


In [3]:
try:
    num_gpus = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))
    print("num_gpus=", num_gpus)
    if num_gpus > 1:
        strategy = tf.distribute.MirroredStrategy()
    else:
        strategy = tf.distribute.OneDeviceStrategy("gpu:0")
except Exception as e:
    print(e)
    print("fallback to CPU")
    strategy = tf.distribute.OneDeviceStrategy("cpu")

num_gpus= 1


In [4]:
#tf.config.experimental_run_functions_eagerly(True)

In [5]:
def dist(A,B):
    na = tf.reduce_sum(tf.square(A), -1)
    nb = tf.reduce_sum(tf.square(B), -1)
 
    na = tf.reshape(na, [tf.shape(na)[0], -1, 1])
    nb = tf.reshape(nb, [tf.shape(na)[0], 1, -1])
    Dsq = tf.clip_by_value(na - 2*tf.linalg.matmul(A, B, transpose_a=False, transpose_b=True) + nb, 1e-12, 1e12)
    D = tf.sqrt(Dsq)
    return D


In [6]:
#Given a list of [Nbatch, Nelem, Nfeat] input nodes, computes the dense [Nbatch, Nelem, Nelem] adjacency matrices
class Distance(tf.keras.layers.Layer):

    def __init__(self, dist_shape, *args, **kwargs):
        super(Distance, self).__init__(*args, **kwargs)

    def call(self, inputs1, inputs2):
        #compute the pairwise distance matrix between the vectors defined by the first two components of the input array
        #inputs1, inputs2: [Nbatch, Nelem, distance_dim] embedded coordinates used for element-to-element distance calculation
        D = dist(inputs1, inputs2)
      
        #adjacency between two elements should be high if the distance is small.
        #this is equivalent to radial basis functions. 
        #self-loops adj_{i,i}=1 are included, as D_{i,i}=0 by construction
        adj = tf.math.exp(-1.0*D)

        #optionally set the adjacency matrix to 0 for low values in order to make the matrix sparse.
        #need to test if this improves the result.
        #adj = tf.keras.activations.relu(adj, threshold=0.01)

        return adj

In [7]:
class InputEncoding(tf.keras.layers.Layer):
    def __init__(self, num_input_classes):
        super(InputEncoding, self).__init__()
        self.num_input_classes = num_input_classes
        
    def call(self, X):
        #X: [Nbatch, Nelem, Nfeat] array of all the input detector element feature data

        #X[:, :, 0] - categorical index of the element type
        Xid = tf.one_hot(tf.cast(X[:, :, 0], tf.int32), self.num_input_classes)

        #X[:, :, 1:] - all the other non-categorical features
        Xprop = X[:, :, 1:]
        return tf.concat([Xid, Xprop], axis=-1)

In [8]:
## Graph Highway network
# https://arxiv.org/pdf/2004.04635.pdf
#https://github.com/gcucurull/jax-ghnet/blob/master/models.py 
class GHConv(tf.keras.layers.Layer):
    def __init__(self, k, *args, **kwargs):
        self.activation = kwargs.pop("activation")
        self.hidden_dim = args[0]
        self.k = k

        super(GHConv, self).__init__(*args, **kwargs)

        self.W_t = self.add_weight(shape=(self.hidden_dim, self.hidden_dim), name="w_t", initializer="random_normal")
        self.b_t = self.add_weight(shape=(self.hidden_dim, ), name="b_t", initializer="zeros")
        self.W_h = self.add_weight(shape=(self.hidden_dim, self.hidden_dim), name="w_h", initializer="random_normal")
        self.theta = self.add_weight(shape=(self.hidden_dim, self.hidden_dim), name="theta", initializer="random_normal")
 
    def call(self, x, adj):
        #compute the normalization of the adjacency matrix
        in_degrees = tf.reduce_sum(adj, axis=-1)
        #add epsilon to prevent numerical issues from 1/sqrt(x)
        norm = tf.expand_dims(tf.pow(in_degrees + 1e-6, -0.5), -1)
        norm_k = tf.pow(norm, self.k)
        adj_k = tf.pow(adj, self.k)

        f_hom = tf.linalg.matmul(x, self.theta)
        f_hom = tf.linalg.matmul(adj_k, f_hom*norm_k)*norm_k

        f_het = tf.linalg.matmul(x, self.W_h)
        gate = tf.nn.sigmoid(tf.linalg.matmul(x, self.W_t) + self.b_t)
        #tf.print(tf.reduce_mean(f_hom), tf.reduce_mean(f_het), tf.reduce_mean(gate))

        out = gate*f_hom + (1-gate)*f_het
        return out

## Simple Graph Conv layer
class SGConv(tf.keras.layers.Dense):
    def __init__(self, k, *args, **kwargs):
        super(SGConv, self).__init__(*args, **kwargs)
        self.k = k
    
    def call(self, inputs, adj):
        W = self.weights[0]
        b = self.weights[1]

        #compute the normalization of the adjacency matrix
        in_degrees = tf.reduce_sum(adj, axis=-1)
        #add epsilon to prevent numerical issues from 1/sqrt(x)
        norm = tf.expand_dims(tf.pow(in_degrees + 1e-6, -0.5), -1)
        norm_k = tf.pow(norm, self.k)

        support = (tf.linalg.matmul(inputs, W))
     
        #k-th power of the normalized adjacency matrix is nearly equivalent to k consecutive GCN layers
        adj_k = tf.pow(adj, self.k)
        out = tf.linalg.matmul(adj_k, support*norm_k)*norm_k

        return self.activation(out + b)

In [9]:
#Simple message passing based on a matrix multiplication
class DNNSuperCluster(tf.keras.Model):
    
    def __init__(self, activation=tf.nn.selu, 
                     hidden_dim_coord=256, hidden_dim_input=256, hidden_dim_id=256,     
                     n_layers_input=2, n_layers_id=3, n_layers_coord=2,
                     distance_dim=256, num_conv=4, convlayer="ghconv", dropout=0.1):
        super(DNNSuperCluster, self).__init__()
        self.activation = activation

        #self.enc = InputEncoding(3)
        
        # layers for distance coordinate extraction
        self.layers_coord = [ ]
        for i in range(n_layers_coord):
            layer_coord_i = tf.keras.layers.Dense(hidden_dim_coord, activation=activation, name="disctcoords_"+str(i))
            self.layers_coord.append(layer_coord_i)

        self.layer_distcoords = tf.keras.layers.Dense(distance_dim, activation="linear", name="distcoords_final")
        self.layer_distance = Distance(distance_dim, name="distance")

        # layers for feature extraction 
        self.layers_input = [ ]
        for i in range(n_layers_input):
            layer_input_i = tf.keras.layers.Dense(hidden_dim_input, activation=activation, name="input_"+str(i))
            layer_input_i_do = tf.keras.layers.Dropout(dropout)
            self.layers_input.append((layer_input_i, layer_input_i_do))
        
       

        # Graph convolutions
        if convlayer == "sgconv":
            self.layer_conv1 = SGConv(num_conv, hidden_dim_input, activation=activation, name="conv1")
            #self.layer_conv2 = SGConv(num_conv, 2*hidden_dim+len(class_labels), activation=activation, name="conv2")
        elif convlayer == "ghconv":
            self.layer_conv1 = GHConv(num_conv, hidden_dim_input, activation=activation, name="conv1")
            #self.layer_conv2 = GHConv(num_conv, 2*hidden_dim+len(class_labels), activation=activation, name="conv2")

        # Output layers
        self.layers_id = [ ]
        for i in range(n_layers_id):
            layer_id_i = tf.keras.layers.Dense(hidden_dim_id, activation=activation, name="id_"+str(i))
            layer_id_i_do = tf.keras.layers.Dropout(dropout)
            self.layers_id.append((layer_id_i, layer_id_i_do))
            
        # binary output logits
        self.layer_id = tf.keras.layers.Dense(1, activation="linear", name="out_id")
        
 
    def predict_distancematrix(self, inputs, training=True):
        x = inputs
        for layer_coord in self.layers_coord:
            x = layer_coord(x)

        distcoords = self.layer_distcoords(x)

        dm = self.layer_distance(distcoords, distcoords)
        
        # masking if the first element is -1
        msk_elem = tf.expand_dims(tf.cast(inputs[:, :, 0] != -1, dtype=tf.float32), -1)
        dm = dm*msk_elem

        return dm

    #@tf.function(input_signature=[tf.TensorSpec(shape=[None, 15], dtype=tf.float32)])
    def call(self, inputs, training=True):
        # separate cluster energies from rescaled inputs
        X = inputs[:,:,1:]
        cl_energies = inputs[:,:,0]
        
        msk_input = tf.expand_dims(tf.cast(X[:, :, 0] != -1, tf.float32), -1)

        dm = self.predict_distancematrix(X, training=training)
        
        x = X
        for layer_input, layer_input_do in self.layers_input:
            x = layer_input(x)
            x = layer_input_do(x, training)
            
        x = self.layer_conv1(x, dm)
        
        for layer_id, layer_id_do in self.layers_id:
            x = layer_id(x)
            x = layer_id_do(x, training)
            
        out_id_logits = self.layer_id(x)
        
        energies = tf.expand_dims(cl_energies, axis=-1)
        # add the cluster energies in the output, in the future we can add here corrections
        output = tf.concat([out_id_logits,energies], axis=-1)
        # mask to 0 the padded output
        output_masked = output * msk_input
        
        #return masked output logits and the predicted total energy
        return output_masked

# Loss definition

In [10]:
#@tf.function
def separate_true(y):
    # one-hot encoding for true label (signal,PU,noise)
    # the padded elements have -1 so they are one_hot to (0,0)
    #y_onehot = tf.one_hot(tf.cast(y[:,1:], tf.int32), nclass_labels)
    ytrue = y[:, 1:]
    true_en = y[:, 0]
    mask = tf.cast(ytrue!=-1, tf.float32)
    ytrue_msk = ytrue * mask
    return ytrue_msk, true_en, mask

#@tf.function
def get_true_mask(y):
    # mask for elements that should be included in supercluster
    in_sc = tf.cast(y[:,1:] == 1., tf.float32)
    # number of padding elements
    padded = tf.reduce_sum(tf.cast(y[:,1:] == -1., tf.float32), axis=-1)
    return in_sc, padded
    
#@tf.function
def separate_pred(ypred):
    ens = ypred[:,:,1]
    ypred_ext = ypred[:,:,0]
    # 0 not include in energy sum, 1 include in energy sum
    # masked elements have pred_id=0 so they do not enter in the energy sum
    predid_mask = tf.cast(tf.math.sigmoid(ypred_ext)[:,:] > 0.5, tf.float32)
    # predicted total energy
    pred_en =  tf.reduce_sum( ens * predid_mask, axis=-1)
    # one-hot encoding for true label (signal,PU,noise)
    return ypred_ext, pred_en, predid_mask
    

In [36]:
#@tf.function
def mse_unreduced(true, pred):
    return tf.math.pow(true-pred,2)

#@tf.function
def msle_unreduced(true, pred):
    return tf.math.pow(tf.math.log(tf.math.abs(true) + 1.0) - tf.math.log(tf.math.abs(pred) + 1.0), 2)


#@tf.function
def my_loss_full(y_true, y_pred):
    y_true_msk, true_en, true_mask = separate_true(y_true)
    y_pred_msk, pred_en, pred_id = separate_pred(y_pred)
    # since the padded y_true is -1 -> it gives [0,0] when it is onehot. The ypred for batched is [0,0] so the loss
    # is automatically 0 for padded samples
    #tf.print(y_true_msk, y_true_msk.shape)
    #tf.print(y_pred_msk, y_true_msk.shape)
    
    # apply mask on loss
    l1 = tf.nn.softmax_cross_entropy_with_logits(y_true_msk, y_pred_msk) * true_mask
    #l1 = tf.keras.backend.binary_crossentropy(y_true_msk, y_pred_msk, from_logits=True) * true_mask
    
    #tf.print(l1)
    # true energy loss
    mask_outsc = tf.cast(true_en == 0., tf.float32)
    mask_insc = tf.cast(true_en != 0., tf.float32)
    n_outsc = tf.reduce_sum(mask_outsc)
    n_insc = tf.reduce_sum(mask_insc)
    
    l2_en = mse_unreduced(true_en, pred_en)
    l2_en_log = msle_unreduced(true_en, pred_en)
    
    # separate mean resolution for windows with Caloparticle or not
    l2_en_outsc = tf.reduce_sum(l2_en * mask_outsc) / n_outsc
    l2_en_insc = tf.reduce_sum(l2_en * mask_insc) / n_insc
    l2_en_outsc_log = tf.reduce_sum(l2_en_log * mask_outsc) / n_outsc
    l2_en_insc_log = tf.reduce_sum(l2_en_log * mask_insc) / n_insc
    
    ltot = 1e4*tf.reduce_mean(l1) + 20* l2_en_insc +   10*l2_en_outsc + 200* l2_en_insc_log  + 100* l2_en_outsc_log
    
    return ltot


In [37]:
def energy_resolution_outsc(y_true, y_pred):
    y_true_msk, true_en, true_mask = separate_true(y_true)
    y_pred_msk, pred_en, pred_id = separate_pred(y_pred)
    mask_outsc = tf.cast(true_en == 0., tf.float32)
    n_outsc = tf.reduce_sum(mask_outsc)
    return tf.reduce_sum(mse_unreduced(true_en, pred_en)*mask_outsc) / n_outsc

def energy_resolution_insc(y_true, y_pred):
    y_true_msk, true_en, true_mask = separate_true(y_true)
    y_pred_msk, pred_en, pred_id = separate_pred(y_pred)
    mask_insc = tf.cast(true_en != 0., tf.float32)
    n_insc = tf.reduce_sum(mask_insc)
    return tf.reduce_sum(mse_unreduced(true_en, pred_en)*mask_insc) / n_insc

def energy_resolution_outsc_log(y_true, y_pred):
    y_true_msk, true_en, true_mask = separate_true(y_true)
    y_pred_msk, pred_en, pred_id = separate_pred(y_pred)
    mask_outsc = tf.cast(true_en == 0., tf.float32)
    n_outsc = tf.reduce_sum(mask_outsc)
    return tf.reduce_sum(msle_unreduced(true_en, pred_en)*mask_outsc) / n_outsc

def energy_resolution_insc_log(y_true, y_pred):
    y_true_msk, true_en, true_mask = separate_true(y_true)
    y_pred_msk, pred_en, pred_id = separate_pred(y_pred)
    mask_insc = tf.cast(true_en != 0., tf.float32)
    n_insc = tf.reduce_sum(mask_insc)
    return tf.reduce_sum(msle_unreduced(true_en, pred_en)*mask_insc) / n_insc

In [38]:
def get_tpfn_metrics(y_true, y_pred):
    y_true_mask, n_padded = get_true_mask(y_true)
    y_false_mask = (tf.cast(y_true_mask == 0., tf.float32))
    
    # pred_id contains the last n_padded elements to 0 that will be always True negatives
    y_pred_onehot, pred_en, pred_id = separate_pred(y_pred)
    
    n_pos = tf.reduce_sum(y_true_mask, axis=-1)
    n_neg = tf.reduce_sum(y_false_mask, axis=-1) - n_padded
    
    n_tot = n_neg + n_pos
    
    true_pos = tf.reduce_sum(pred_id * y_true_mask, axis=-1)
    false_neg = n_pos - true_pos
    
    false_pos = tf.reduce_sum(pred_id * y_false_mask, axis=-1)
    true_neg = n_neg - false_pos
    
    return n_tot, true_pos, false_neg, false_pos, true_neg, 

In [39]:
def precision(tp,tn,fp,fn):
    return tp/(tp+fp)

def recall(tp,tn,fp,fn):
    return tp/(tp+fn)

def accuracy(tp,tn,fp,fn):
    return (tp+tn)/(tp+tn+fp+fn)

In [40]:
class Precision(tf.keras.metrics.Metric):

    def __init__(self, name='precision', **kwargs):
        super(Precision, self).__init__(name=name, **kwargs)
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        n_tot, true_pos, false_neg, false_pos, true_neg = get_tpfn_metrics(y_true, y_pred)
        self.tp.assign_add(tf.reduce_sum(true_pos))
        self.fp.assign_add(tf.reduce_sum(false_pos))

    def result(self):
        return self.tp / (self.tp + self.fp)

    def reset_states(self):
        self.tp.assign(0)
        self.fp.assign(0)
        
class Recall(tf.keras.metrics.Metric):

    def __init__(self, name='recall', **kwargs):
        super(Recall, self).__init__(name=name, **kwargs)
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        n_tot, true_pos, false_neg, false_pos, true_neg = get_tpfn_metrics(y_true, y_pred)
        self.tp.assign_add(tf.reduce_sum(true_pos))
        self.fn.assign_add(tf.reduce_sum(false_neg))

    def result(self):
        return self.tp / (self.tp + self.fn)

    def reset_states(self):
        self.tp.assign(0)
        self.fn.assign(0)


### Settings

In [41]:
data_path = "/storage/ECAL/training_data/window_data/electrons/recordio_v4"
models_path = "/storage/ECAL/deepcluster/models/gcn_models_v9/"

#rain_steps_per_epoch = 
#eval_steps_per_epoch = 3e5 // batch_size
from collections import namedtuple
Args = namedtuple('args', [ 'models_path', 'load','nepochs','ntrain','nval','nfeatures',
                            'n_seed_features','batch_size','lr_decay','lr',
                            'hidden_dim_input','hidden_dim_coord', 'hidden_dim_id',
                            'n_layers_input', 'n_layers_id', 'n_layers_coord',
                           'distance_dim','num_conv','dropout','convlayer',
                           'opt'])

args = Args( 
models_path = models_path,
load = False,
nepochs = 100,
ntrain = 500000,
nval = 100000,
nfeatures = 13,
n_seed_features = 12,
lr_decay = 0,
lr = 0.00001,
batch_size = 150,
n_layers_input = 3,
n_layers_id = 2,
n_layers_coord = 2,
hidden_dim_input = 100,
hidden_dim_coord = 50,
hidden_dim_id = 100,
distance_dim = 30,
num_conv = 2,
dropout = 0.01,
convlayer = 'ghconv',
opt='adam'
        )

### Dataset loading

In [42]:
def scale_features_clusters(X):
    '''
    'is_seed',"cluster_deta", "cluster_dphi", "en_cluster", "et_cluster",
    "cl_f5_r9", "cl_f5_sigmaIetaIeta","cl_f5_sigmaIetaIphi","cl_f5_sigmaIphiIphi",
    "cl_f5_swissCross", "cl_nxtals", "cl_etaWidth", "cl_phiWidth
    '''
    x_mean = tf.constant( 
        [   0.,  -7.09402501e-04, -1.27142875e-04,  1.30375508e+00,  5.67249500e-01, 
            1.92096066e+00,  1.31476120e-02,  1.62948213e-05,  1.42948806e-02,
            5.92920497e-01,  1.49597644e+00,  3.36213188e-03,  3.06446267e-03]
        )

    x_scale = tf.constant(
        [  1.,  1.10279784e-01, 3.30488055e-01, 2.62605247e+00, 1.16284769e+00,
            7.81094814e+00, 1.70392176e-02, 3.05995567e-04, 1.80176053e-02,
            1.99316624e+00, 1.88845046e+00, 4.12315715e-03, 4.79639033e-03]       
        )
    return (X-x_mean)/ x_scale

def scale_features_seed(X):
    '''
     "seed_eta", "seed_iz","en_seed","et_seed",
     "seed_f5_r9", "seed_f5_sigmaIetaIeta","seed_f5_sigmaIetaIphi","seed_f5_sigmaIphiIphi",
     "seed_f5_swissCross","seed_nxtals", "seed_etaWidth", "seed_phiWidth",
    '''
    x_mean = tf.constant( 
        [   6.84241156e-03,  1.62242679e-03,  5.81495577e+01,  2.57215845e+01, 
            1.00772582e+00,  1.35803461e-02, -4.29317013e-06,  1.71072024e-02,
            4.90466869e-01,  5.10511982e+00,  8.82101138e-03,  1.04095965e-02 ]
    
        )

    x_scale = tf.constant(
        [   1.31333380e+00, 5.06988411e-01, 9.21157365e+01, 2.98580765e+01, 
            1.17047757e-01, 1.11969442e-02, 1.86572967e-04, 1.31036359e-02,
            4.01511744e-01, 5.67007350e+00, 6.14304203e-03, 7.24808860e-03]       
        )
    return (X-x_mean)/ x_scale

In [43]:
def _parse_tfr_element(element):
    parse_dic = {
        'X':      tf.io.FixedLenFeature([], tf.string),
        'X_seed': tf.io.FixedLenFeature([], tf.string),
        'y':      tf.io.FixedLenFeature([], tf.string),
        'n_clusters': tf.io.FixedLenFeature([], tf.int64)
    }
    example_message = tf.io.parse_single_example(element, parse_dic)

    X = example_message['X']
    X_seed = example_message['X_seed']
    y = example_message['y']
    nclusters = example_message['n_clusters']
    
    arr_X = tf.io.parse_tensor(X, out_type=tf.float32)
    arr_X_seed = tf.io.parse_tensor(X_seed, out_type=tf.float32)
    arr_y = tf.io.parse_tensor(y, out_type=tf.float32)
    
    #https://github.com/tensorflow/tensorflow/issues/24520#issuecomment-577325475
    arr_X.set_shape(     tf.TensorShape((None, args.nfeatures)))
    arr_X_seed.set_shape(tf.TensorShape((1, args.n_seed_features)))
    arr_y.set_shape(     tf.TensorShape((None,)))
 
    return arr_X, arr_X_seed, nclusters, arr_y
  
def _stack_seed_features(arr_X, arr_X_seed, nclusters, arr_y):
    en_clusters = tf.expand_dims(arr_X[:,3], axis=-1)
    rescaled_X = scale_features_clusters(arr_X)
    rescaled_X_seed = scale_features_seed(arr_X_seed)
    X = tf.concat([en_clusters, rescaled_X, tf.broadcast_to(rescaled_X_seed,[nclusters,rescaled_X_seed.shape[1]] )],
                  axis=1)
    return X,arr_y

In [44]:
# padding shape
ps = ([None,args.nfeatures+args.n_seed_features+1],[None,])

# Create datasets from TFRecord files.
dataset = tf.data.TFRecordDataset(tf.io.gfile.glob('{}/training-*'.format(data_path)))
dataset = dataset.map(_parse_tfr_element,num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(_stack_seed_features,num_parallel_calls=tf.data.experimental.AUTOTUNE) # deterministic=False in TFv2.3
dataset = dataset.shuffle(10000, reshuffle_each_iteration=True)

ds_train = dataset.take(args.ntrain).padded_batch(args.batch_size, padded_shapes=ps, drop_remainder=True,padding_values=(-1.,-1.))
ds_test = dataset.skip(args.ntrain).take(args.nval).padded_batch(args.batch_size, padded_shapes=ps, drop_remainder=True,padding_values=(-1.,-1.))

ds_train_r = ds_train.repeat(args.nepochs)
ds_test_r = ds_test.repeat(args.nepochs)


In [45]:
idata = iter(dataset)

In [46]:
d = next(idata)
d

(<tf.Tensor: shape=(15, 26), dtype=float32, numpy=
 array([[ 2.60765910e+00,  1.00000000e+00,  6.43275212e-03,
          3.84712475e-04,  4.96526301e-01,  7.15929449e-01,
         -1.17906384e-01, -7.71608949e-01, -5.32518215e-02,
         -7.93384075e-01,  2.04237625e-01,  1.32596731e+00,
         -1.09502457e-01, -2.92296886e-01,  9.34330761e-01,
         -3.20012611e-03, -6.02957785e-01, -8.14580858e-01,
         -6.60058334e-02, -1.21286190e+00,  2.30106749e-02,
         -1.30553091e+00,  1.26903677e+00, -1.94903940e-01,
         -9.62125242e-01, -1.20681524e+00],
        [ 2.60415387e+00,  0.00000000e+00, -1.46129057e-01,
          1.31661534e+00,  4.95191514e-01,  7.31447101e-01,
         -1.55343652e-01, -7.71608949e-01, -5.32518215e-02,
         -3.27732670e-03, -2.70530909e-01, -2.62636721e-01,
         -1.17125720e-01,  2.26948810e+00,  9.34330761e-01,
         -3.20012611e-03, -6.02957785e-01, -8.14580858e-01,
         -6.60058334e-02, -1.21286190e+00,  2.30106749e-02,
     

In [47]:
def get_unique_run():
    previous_runs = os.listdir(args.models_path)
    if len(previous_runs) == 0:
        run_number = 1
    else:
        run_number = max([int(s.split('run_')[1]) for s in previous_runs]) + 1
    return run_number

In [48]:
if args.lr_decay > 0:
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            args.lr,
            decay_steps=2*int(args.ntrain//args.batch_size),
            decay_rate=args.lr_decay
        )
else:
    lr_schedule = args.lr


In [49]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model = DNNSuperCluster(hidden_dim_input=args.hidden_dim_input,hidden_dim_coord=args.hidden_dim_coord,
                            hidden_dim_id=args.hidden_dim_id, 
                            n_layers_input=args.n_layers_input, n_layers_id=args.n_layers_id, n_layers_coord=args.n_layers_coord,
                            distance_dim=args.distance_dim, 
                            num_conv=args.num_conv, convlayer=args.convlayer, dropout=args.dropout)
   

In [50]:
if not os.path.isdir(args.models_path):
    os.makedirs(args.models_path)

name =  'run_{:02}'.format(get_unique_run())

outdir = args.models_path + name

if os.path.isdir(outdir):
    print("Output directory exists: {}".format(outdir), file=sys.stderr)

print(outdir)

/storage/ECAL/deepcluster/models/gcn_models_v9/run_08


In [51]:
callbacks = []
tb = tf.keras.callbacks.TensorBoard(
    log_dir=outdir, histogram_freq=2, 
    write_graph=False, 
    write_images=True,
    update_freq='epoch',
    profile_batch=0,
)
tb.set_model(model)
callbacks += [tb]

terminate_cb = tf.keras.callbacks.TerminateOnNaN()
callbacks += [terminate_cb]

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=outdir + "/weights.{epoch:02d}-{val_loss:.6f}.hdf5",
    save_weights_only=True,
    verbose=0
)
cp_callback.set_model(model)
callbacks += [cp_callback]

loss_fn = my_loss_full



In [52]:
with strategy.scope():
    model.compile(optimizer=args.opt, loss=loss_fn,
        metrics=[Precision(),Recall(), energy_resolution_insc,energy_resolution_outsc,
                     energy_resolution_insc_log,energy_resolution_outsc_log,])
   
    for X, y in ds_train:
        ypred = model(X)
        l = loss_fn(y, ypred)
        break

InvalidArgumentError: Incompatible shapes: [150] vs. [150,20] [Op:Mul] name: mul/

In [ ]:
yoh,true_en,true_mask = separate_true(y)

In [53]:
model.summary()

Model: "dnn_super_cluster_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
disctcoords_0 (Dense)        multiple                  1300      
_________________________________________________________________
disctcoords_1 (Dense)        multiple                  2550      
_________________________________________________________________
distcoords_final (Dense)     multiple                  1530      
_________________________________________________________________
distance (Distance)          multiple                  0         
_________________________________________________________________
input_0 (Dense)              multiple                  2600      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
input_1 (Dense)              multiple          

In [54]:
if args.load:
    #ensure model input size is known
    for X, y in ds_train:
        model(X)
        break

    model.load_weights(args.load)
if args.nepochs > 0:
    ret = model.fit(ds_train_r,
        validation_data=ds_test_r, epochs=args.nepochs,
        steps_per_epoch=args.ntrain//args.batch_size, validation_steps=args.nval//args.batch_size,
        verbose=True,
        callbacks=callbacks
    )

Train for 3333 steps, validate for 666 steps
Epoch 1/100
   1/3333 [..............................] - ETA: 3:36:03

KeyError: 'val_loss'

In [ ]:
str(args)

In [ ]:
with open(outdir + "/args.txt",'w') as config:
    config.write(str(args))
    

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()